# Preprocesamiento de Texto

## Introducción

El preprocesamiento de texto es el primer paso para trabajar con datos textuales en proyectos de Procesamiento de Lenguaje Natural (NLP). Antes de aplicar modelos o algoritmos, es necesario transformar los textos en un formato más estructurado y limpio que permita analizarlos con mayor facilidad. Este proceso ayuda a reducir el ruido en los datos y resaltar lo que es realmente útil para las tareas posteriores.

En este cuaderno vamos a explorar técnicas básicas y avanzadas de preprocesamiento aplicadas a un texto literario, utilizando un fragmento de "Cien Años de Soledad". El objetivo es aprender a preparar los datos para tareas como clasificación, análisis de sentimiento o generación de texto.

### ¿Qué veremos?

1. **Normalización del texto**: Minúsculas, eliminación de caracteres innecesarios y formatos consistentes.
2. **Tokenización**: Separar el texto en palabras o frases.
3. **Eliminación de palabras comunes (stopwords)**: Filtrar palabras que no aportan información útil.
4. **Stemming y lematización**: Simplificar palabras a su raíz o forma base.
5. **Representación numérica**: Convertir el texto en vectores para análisis cuantitativo.


In [1]:
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
import spacy ## Procesamiento de lenguaje natural

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion

############## Es necesario descargar estos recursos para poder ejecutar el script ################

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## 2. Cargar el texto

Para este ejercicio inicial usaremos un texto sencillo, el cual se cargará en una variable.

In [3]:
texto = """Hace varios años, en el pelotón de fusilamiento, el coronel Aureliano Buendía había de recordar aquella tarde remota en que su padre lo llevó a conocer el hielo. """

In [2]:
## En el caso de un corpus se puede hacer a tra ves de un csv
import pandas as pd

Corpus_DF=pd.read_csv('https://raw.githubusercontent.com/Izainea/nlp_ean/refs/heads/main/Datos/Datos%20Crudos/corpus_chismofilias.csv')
Corpus_DF

,Comentario,Sentimiento
0,"Me encanta usar las chismofilias, ahora estoy ...",Positivo
1,"Es increíble, escucho cosas que nunca imaginé....",Positivo
2,"La calidad del audio es sorprendente, puedo es...",Positivo
3,"La batería dura mucho tiempo, lo uso todo el d...",Positivo
4,"No puedo creer que exista algo así, es como te...",Positivo
5,"El diseño es muy discreto, nadie sospecha que ...",Positivo
6,"El alcance es limitado, no puedo escuchar lo q...",Negativo
7,"Es muy caro para lo que ofrece, esperaba más f...",Negativo
8,A veces se desconecta y pierdo lo que estaban ...,Negativo
9,"No cumple con las expectativas, el sonido es m...",Negativo


## 3. Normalización

La **normalización de texto** se refiere al proceso de transformar un texto para que tenga un formato uniforme y consistente. Esto puede incluir convertir a minúsculas, eliminar caracteres especiales, números o espacios adicionales. Su principal objetivo es reducir la complejidad y estandarizar el texto antes de aplicar técnicas de análisis.

### ¿Cuándo usar la normalización?

La normalización es útil en varios contextos, pero no siempre es necesaria. Aquí te dejo una guía sobre cuándo es apropiado aplicarla:

#### **1. Modelos tradicionales de NLP**
- **Modelos como Bag of Words (BoW), TF-IDF o Word2Vec**:
  - Estos enfoques se benefician de textos más limpios y homogéneos, ya que cualquier variación (como diferencias en el uso de mayúsculas) puede aumentar innecesariamente la dimensionalidad.
  - Ejemplo: `Casa` y `casa` serán consideradas distintas si no se normaliza el texto.

#### **2. Análisis de frecuencia o clustering**
- En tareas como la identificación de palabras más comunes, búsqueda de temas o clustering de textos, la normalización es esencial para evitar que caracteres especiales o inconsistencias afecten los resultados.

#### **3. Modelos avanzados como LLMs (e.g., GPT, BERT)**
- Los modelos de lenguaje de gran tamaño ya incluyen mecanismos integrados para manejar textos sin normalizar. **No es necesario preprocesar demasiado**:
  - Estos modelos tienen vocabularios entrenados que incluyen variaciones comunes como mayúsculas o caracteres especiales.
  - Sin embargo, en algunos casos, eliminar información irrelevante (como emojis o HTML) puede ser beneficioso dependiendo del dominio de los datos.

### **¿Qué pasos incluye la normalización?**

#### 1. **Conversión a minúsculas**
   - Utilidad: Evita que `Casa` y `casa` se traten como diferentes.
   - Ejemplo: `"Hola Mundo"` → `"hola mundo"`

#### 2. **Eliminación de caracteres especiales**
   - Utilidad: Simplifica el análisis al eliminar puntuación, emojis o símbolos irrelevantes.
   - Ejemplo: `"¡Hola! ¿Cómo estás?"` → `"Hola Como estas"`

#### 3. **Eliminación de números (opcional)**
   - Utilidad: En textos narrativos o sentimentales, los números suelen no ser informativos.
   - Ejemplo: `"En el año 2023"` → `"En el año"`

#### 4. **Eliminación de espacios adicionales**
   - Utilidad: Limpia la estructura del texto.
   - Ejemplo: `"Hola     Mundo"` → `"Hola Mundo"`

### **¿Cuándo evitar la normalización?**

- **Textos donde las mayúsculas son significativas**: En análisis de sentimiento, una frase como `"¡GRACIAS!"` puede tener un impacto emocional diferente a `"gracias"`.
- **Datos con emojis o símbolos relevantes**: En algunos dominios, eliminar emojis podría perder información clave.
- **Entradas para LLMs**: Estos modelos ya gestionan caracteres especiales y capitalización, por lo que normalizar excesivamente podría eliminar información útil.

<img src="https://www.mermaidchart.com/raw/9218311f-e863-4e37-824a-b23305925151?theme=light&version=v0.1&format=svg" alt="Diagrama de flujo para la normalización" style="width:100%; max-width:800px;">

In [ ]:
############## Normalización ################

## Convertir a minusculas

texto = texto.lower()

## Eliminar caracteres especiales

texto = re.sub(r"[\W_]+", " ", texto) ## Elimina caracteres especiales y numeros la expresion regular \W es para caracteres especiales y \d para numeros, todo lo que no sea una letra se reemplaza por un espacio.



In [8]:
# prompt: Crea una historia donde nombres a muchas personas de habla hispana
historia = """
  En el corazón de la vibrante ciudad de Medellín, vivía una joven llamada Camila.
  Ella soñaba con ser escritora y pasaba horas leyendo libros de Gabriel García Márquez.
  Su mejor amigo, Santiago, era un talentoso músico que componía melodías conmovedoras.
  Juntos, compartían tardes de café con su vecina, una abuela llamada Elena, quien contaba historias de su juventud en la isla de Cuba.
  Un día, un misterioso hombre llamado Javier llegó al barrio. Era un artista enigmático que pintaba retratos conmovedores.
  Su llegada coincidió con la visita de una familia de México, integrada por Ricardo, su esposa Sofia y sus hijos, Isabella y Miguel.
  Camila y Santiago se hicieron amigos de ellos y organizaron una fiesta en la que todos pudieron compartir sus culturas y tradiciones.
  Se reunieron alrededor de una mesa llena de sabrosos platillos como tamales, arepas y empanadas.
  Isabella, con su gran talento para el baile, aprendió pasos de salsa de manos de un alegre joven llamado Alejandro.
  Miguel, un entusiasta de la naturaleza, pasaba horas explorando los jardines junto a Natalia, una bióloga que vivía en la ciudad.
  Mientras tanto, Elena seguía contando historias sobre sus viajes a Puerto Rico y Venezuela, fascinando a todos con sus relatos.
  En esa época de encuentros y nuevas amistades, Javier encontró la inspiración para su próxima obra,
  un retrato colectivo que reflejaba la diversidad y la calidez de la comunidad.
  Todos, Camila, Santiago, Elena, Javier, Ricardo, Sofia, Isabella, Miguel, Alejandro, y Natalia,
  se convirtieron en una familia extendida, unidos por la música, la literatura, y el arte.
  """

re.findall(r'M[a-záéíóú][a-záéíóú]+', historia)

['Medellín', 'Márquez', 'México', 'Miguel', 'Miguel', 'Mientras', 'Miguel']

In [ ]:
# prompt: Has una historia donde nombro muchos correos electrónicos y telefonos

historia = """
  En el corazón de la vibrante ciudad de Medellín, vivía una joven llamada Camila.
  Ella soñaba con ser escritora y pasaba horas leyendo libros de Gabriel García Márquez.
  Su mejor amigo, Santiago, era un talentoso músico que componía melodías conmovedoras.
  Juntos, compartían tardes de café con su vecina, una abuela llamada Elena, quien contaba historias de su juventud en la isla de Cuba.
  Un día, un misterioso hombre llamado Javier llegó al barrio. Era un artista enigmático que pintaba retratos conmovedores.
  Su llegada coincidió con la visita de una familia de México, integrada por Ricardo, su esposa Sofia y sus hijos, Isabella y Miguel.
  Camila y Santiago se hicieron amigos de ellos y organizaron una fiesta en la que todos pudieron compartir sus culturas y tradiciones.
  Se reunieron alrededor de una mesa llena de sabrosos platillos como tamales, arepas y empanadas.
  Isabella, con su gran talento para el baile, aprendió pasos de salsa de manos de un alegre joven llamado Alejandro.
  Miguel, un entusiasta de la naturaleza, pasaba horas explorando los jardines junto a Natalia, una bióloga que vivía en la ciudad.
  Mientras tanto, Elena seguía contando historias sobre sus viajes a Puerto Rico y Venezuela, fascinando a todos con sus relatos.
  En esa época de encuentros y nuevas amistades, Javier encontró la inspiración para su próxima obra,
  un retrato colectivo que reflejaba la diversidad y la calidez de la comunidad.
  Todos, Camila, Santiago, Elena, Javier, Ricardo, Sofia, Isabella, Miguel, Alejandro, y Natalia,
  se convirtieron en una familia extendida, unidos por la música, la literatura, y el arte.
  Camila: camila.lopez@gmail.com, +57 300 123 4567
  Santiago: santiago.gomez@hotmail.com, +57 310 987 6543
  Elena: elena.rodriguez@outlook.com, +57 315 111 2222
  Javier: javier.perez@yahoo.com, +57 318 333 4444
  Ricardo: ricardo.martinez@gmail.com, +52 55 1111 2222
  Sofia: sofia.hernandez@hotmail.com, +52 55 3333 4444
  Isabella: isabella.garcia@outlook.com, +52 55 5555 6666
  Miguel: miguel.lopez@yahoo.com, +52 55 7777 8888
  Alejandro: alejandro.sanchez@gmail.com, +57 320 999 0000
  Natalia: natalia.gomez@hotmail.com, +57 311 888 7777
"""

# Expresiones regulares para encontrar correos electrónicos y teléfonos
emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', historia)
telefonos = re.findall(r'\+\d{1,3} \d{2,3} \d{3} \d{4}', historia)

print("Correos electrónicos encontrados:")
for email in emails:
  print(email)

print("\nNúmeros de teléfono encontrados:")
for telefono in telefonos:
telefono

In [ ]:
### Ahora hagamos ese proceso de normalización para todo el corpus

## Convert

Corpus_DF['Comentario Limpio'] = Corpus_DF['Comentario'].str.lower()

## Eliminar caracteres especiales

Corpus_DF['Comentario Limpio'] = Corpus_DF['Comentario Limpio'].apply(lambda x: re.sub(r"[\W_]+", " ", x))

Corpus_DF


,Comentario,Sentimiento,Comentario Limpio
0,"Me encanta usar las chismofilias, ahora estoy ...",Positivo,me encanta usar las chismofilias ahora estoy a...
1,"Es increíble, escucho cosas que nunca imaginé....",Positivo,es increíble escucho cosas que nunca imaginé m...
2,"La calidad del audio es sorprendente, puedo es...",Positivo,la calidad del audio es sorprendente puedo esc...
3,"La batería dura mucho tiempo, lo uso todo el d...",Positivo,la batería dura mucho tiempo lo uso todo el dí...
4,"No puedo creer que exista algo así, es como te...",Positivo,no puedo creer que exista algo así es como ten...
5,"El diseño es muy discreto, nadie sospecha que ...",Positivo,el diseño es muy discreto nadie sospecha que e...
6,"El alcance es limitado, no puedo escuchar lo q...",Negativo,el alcance es limitado no puedo escuchar lo qu...
7,"Es muy caro para lo que ofrece, esperaba más f...",Negativo,es muy caro para lo que ofrece esperaba más fu...
8,A veces se desconecta y pierdo lo que estaban ...,Negativo,a veces se desconecta y pierdo lo que estaban ...
9,"No cumple con las expectativas, el sonido es m...",Negativo,no cumple con las expectativas el sonido es ma...


### Tokenización

La **tokenización** es el proceso de dividir un texto en unidades más pequeñas llamadas **tokens**. Estos pueden ser palabras, frases o incluso caracteres individuales, dependiendo del enfoque. Este paso es fundamental en Procesamiento de Lenguaje Natural (NLP) porque transforma el texto en fragmentos manejables que los algoritmos pueden analizar y procesar.

---

### ¿Por qué es importante la tokenización?

1. **Transformación estructurada**:
   - Convierte el texto no estructurado en datos que pueden procesarse de manera computacional.
   - Ejemplo:
     ```python
     texto = "El gato está sobre la mesa."
     tokens = ["El", "gato", "está", "sobre", "la", "mesa"]
     ```

2. **Preparación para análisis posterior**:
   - Muchos algoritmos de NLP, como Bag of Words (BoW) o TF-IDF, requieren que el texto esté tokenizado para calcular frecuencias o relaciones entre palabras.

3. **Reducción de complejidad**:
   - Ayuda a simplificar problemas complejos al dividirlos en componentes más pequeños y manejables.

4. **Facilita el preprocesamiento**:
   - La tokenización es un paso previo a tareas como la eliminación de palabras vacías (stopwords), stemming, lematización o creación de embeddings.

---

### ¿Cómo se realiza la tokenización?

Existen diversas técnicas de tokenización, desde las más simples hasta las más avanzadas:

1. **Basada en espacios**:
   - Divide las palabras donde hay espacios.
   - Ejemplo: `"Hola, mundo"` → `["Hola,", "mundo"]`
   - Problema: Incluye signos de puntuación como parte de los tokens.

2. **Basada en expresiones regulares**:
   - Usa patrones para identificar palabras, números o caracteres específicos.
   - Ejemplo: Eliminando puntuación:
     ```python
     import re
     texto = "Hola, mundo."
     tokens = re.findall(r'\w+', texto)  # ["Hola", "mundo"]
     ```

3. **Tokenización de subpalabras**:
   - Divide las palabras en fragmentos más pequeños (subword units), como en BERT o WordPiece.
   - Ejemplo:
     - Palabra: `"corriendo"`
     - Tokens: `["corr", "iendo"]`
   - Ventaja: Maneja vocabularios más pequeños y es más robusto frente a palabras desconocidas.

4. **Tokenización basada en caracteres**:
   - Divide el texto en caracteres individuales.
   - Útil en tareas que analizan patrones a nivel de carácter.

---

### Tokenización en modelos avanzados como BERT

Modelos avanzados como **BERT, GPT o RoBERTa** tienen sus propias estrategias de tokenización diseñadas para maximizar la eficiencia y la cobertura del vocabulario:

1. **Vocabulario fijo**:
   - Utilizan un vocabulario limitado, generado previamente, que incluye palabras comunes y fragmentos de palabras.
   - Ejemplo: `"imposible"` → `["im", "pos", "ible"]`

2. **Manejo de palabras desconocidas**:
   - Si una palabra no está en el vocabulario, el modelo la divide en subpalabras para asegurar su representación.

3. **Codificación específica (WordPiece, Byte-Pair Encoding)**:
   - Estas técnicas dividen las palabras en subword units basadas en frecuencias observadas durante el entrenamiento.
   - Ejemplo: `"autonomía"` podría ser dividida en `["auto", "nomía"]`.

4. **Preservación de contexto**:
   - La tokenización específica permite que estos modelos capturen relaciones entre palabras y subpalabras, mejorando el análisis semántico.

---

### ¿Cuándo usar tokenización personalizada?

1. **Modelos preentrenados (e.g., BERT, GPT)**:
   - **No necesitas tokenización manual**. Estos modelos ya incluyen su propio tokenizador, optimizado para su arquitectura y vocabulario.
   - Ejemplo con el tokenizador de BERT:
     ```python
     from transformers import BertTokenizer
     tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
     tokens = tokenizer.tokenize("El gato está sobre la mesa.")
     print(tokens)  # ['el', 'gato', 'está', 'sobre', 'la', 'mesa', '.']
     ```

2. **Modelos tradicionales (TF-IDF, BoW)**:
   - Aquí se recomienda una tokenización básica o basada en expresiones regulares.

3. **Textos en lenguajes complejos (e.g., Chino, Japonés)**:
   - La tokenización puede requerir herramientas específicas como **Jieba** o **MeCab** para manejar caracteres que no están separados por espacios.

---


La tokenización es ¡importantísima! en NLP porque permite estructurar el texto para tareas posteriores. Sin embargo, **la forma de tokenizar depende del modelo y el problema**:

- **Modelos tradicionales** requieren una tokenización explícita.
- **Modelos avanzados como BERT** incluyen su propio tokenizador optimizado, lo que simplifica el preprocesamiento.


<img src="https://www.mermaidchart.com/raw/c333babf-e00a-4830-9c01-ec02d8e4b8aa?theme=light&version=v0.1&format=svg" alt="Diagrama de flujo para la normalización" style="width:100%; max-width:1200px;">

In [ ]:
############## Tokenizacion ################

## Tokenizador por palabras

texto_tokenizado = word_tokenize(texto)

### La funcion word_tokenize de nltk permite tokenizar un texto en palabras, por detrás esta utilizando expresiones regulares para separar las palabras.

texto_tokenizado

['hace',
 'varios',
 'años',
 'en',
 'el',
 'pelotón',
 'de',
 'fusilamiento',
 'el',
 'coronel',
 'aureliano',
 'buendía',
 'había',
 'de',
 'recordar',
 'aquella',
 'tarde',
 'remota',
 'en',
 'que',
 'su',
 'padre',
 'lo',
 'llevó',
 'a',
 'conocer',
 'el',
 'hielo']

In [ ]:
## Tokenizador por expresiones regulares

tokenizer = RegexpTokenizer(r'\w+')
texto_tokenizado = tokenizer.tokenize(texto)

### La clase RegexpTokenizer de nltk permite tokenizar un texto utilizando expresiones regulares, en este caso se tokeniza por palabras.

texto_tokenizado

['hace',
 'varios',
 'años',
 'en',
 'el',
 'pelotón',
 'de',
 'fusilamiento',
 'el',
 'coronel',
 'aureliano',
 'buendía',
 'había',
 'de',
 'recordar',
 'aquella',
 'tarde',
 'remota',
 'en',
 'que',
 'su',
 'padre',
 'lo',
 'llevó',
 'a',
 'conocer',
 'el',
 'hielo']

## 5. Eliminación de stopwords


Las **stop words** son palabras que tienen poco o ningún valor semántico en un análisis de texto. Estas palabras suelen ser artículos, preposiciones, conjunciones o pronombres que, aunque son esenciales para construir frases en un lenguaje natural, no aportan información relevante para el análisis de contenido en tareas específicas.

---

### ¿Por qué eliminar las stop words?

1. **Reducción del ruido en los datos**:
   - Elimina palabras comunes que no diferencian un texto de otro, como "el", "de", "y", "pero".
   - Ejemplo:
     ```python
     texto = "El gato está en la casa y duerme."
     sin_stopwords = "gato casa duerme"
     ```

2. **Optimización de los modelos**:
   - Reduce la dimensionalidad del vocabulario, mejorando la eficiencia computacional.
   - Ayuda a concentrar el análisis en palabras clave más relevantes.

3. **Relevancia en tareas específicas**:
   - En tareas como clasificación de texto o clustering, las stop words pueden diluir patrones importantes.

---

### ¿Cuándo evitar eliminarlas?

Aunque generalmente es útil, no siempre es adecuado eliminar stop words. Aquí hay casos en los que podrían ser importantes:

1. **Análisis de estilo o escritura**:
   - En análisis literarios o de autoría, las stop words pueden reflejar patrones estilísticos.
   - Ejemplo: En textos de Gabriel García Márquez, el uso de "y" es característico de sus frases largas y descriptivas.

2. **Modelos avanzados (e.g., BERT, GPT)**:
   - Los modelos de lenguaje de gran tamaño ya manejan las stop words en su contexto semántico.
   - Eliminarlas podría romper relaciones útiles en tareas como análisis de sentimiento o traducción.

3. **Análisis sintáctico**:
   - Si el objetivo es analizar estructuras gramaticales, las stop words son esenciales.

---

### ¿Cómo identificar y eliminar stop words?

1. **Usando listas predefinidas**:
   - Librerías como `nltk` o `spaCy` incluyen listas de stop words comunes.
   - Ejemplo con `nltk`:
     ```python
     from nltk.corpus import stopwords
     from nltk.tokenize import word_tokenize
     
     nltk.download('stopwords')
     nltk.download('punkt')

     texto = "El gato está en la casa y duerme."
     palabras = word_tokenize(texto.lower())
     stop_words = set(stopwords.words('spanish'))

     sin_stopwords = [palabra for palabra in palabras if palabra not in stop_words]
     print(sin_stopwords)  # ['gato', 'casa', 'duerme']
     ```

2. **Personalizando las listas**:
   - Puedes agregar o quitar palabras según el dominio del texto.
   - Ejemplo: En textos legales, palabras como "ley", "artículo", "reglamento" no deben eliminarse.

3. **Usando expresiones regulares**:
   - Filtrar palabras que cumplen ciertos patrones.
   - Ejemplo: Eliminación de palabras de 1-2 caracteres:
     ```python
     import re
     texto = "El gato está en la casa y duerme."
     palabras = re.findall(r'\b\w{3,}\b', texto.lower())
     print(palabras)  # ['gato', 'está', 'casa', 'duerme']
     ```

---

### Alternativas a eliminar stop words

En algunos casos, en lugar de eliminarlas, puedes:
1. **Ponderarlas**:
   - Usar métodos como TF-IDF que reducen automáticamente el peso de palabras muy frecuentes.

2. **Mantenerlas para tareas contextuales**:
   - En análisis de sentimiento o detección de sarcasmo, las stop words pueden ser significativas.

---

La eliminación de stop words es un paso clave en la mayoría de las tareas de NLP, pero debe aplicarse con criterio:

- **Útil** en modelos tradicionales como TF-IDF o Word2Vec.
- **No siempre necesario** en modelos avanzados como BERT o GPT.
- **Personalizable** según el dominio y los objetivos del análisis.

<img src="https://www.mermaidchart.com/raw/6c157501-141d-40ae-9295-f991bc567f6e?theme=light&version=v0.1&format=svg" alt="Stop Words" style="width:100%; max-width:1200px;">

In [ ]:
############## Eliminación de palabras vacias ################

## Palabras vacias en español

stopwords_esp = stopwords.words('spanish')

## Eliminar palabras vacias

texto_filtrado = [palabra for palabra in texto_tokenizado if palabra not in stopwords_esp]

### La lista stopwords.words('spanish') contiene las palabras vacias en español, se filtran las palabras vacias del texto tokenizado.

stopwords_esp

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [ ]:
texto_filtrado

['hace',
 'varios',
 'años',
 'pelotón',
 'fusilamiento',
 'coronel',
 'aureliano',
 'buendía',
 'recordar',
 'aquella',
 'tarde',
 'remota',
 'padre',
 'llevó',
 'conocer',
 'hielo']

## 6. Lematización o stemming

Tanto la **lemmatización** como el **stemming** son técnicas utilizadas en el preprocesamiento de texto para reducir las palabras a su forma base. Sin embargo, ambas tienen diferencias importantes en su propósito y en cómo se implementan.

---

### Stemming

El **stemming** consiste en reducir una palabra a su raíz o **stem**, eliminando afijos como sufijos y prefijos. Este proceso no garantiza que la raíz sea una palabra válida en el lenguaje, ya que se basa en reglas lingüísticas simples.

#### Ejemplo:
- Palabras: "corriendo", "corrió", "correr"
- Resultado del stemming: "corr"

#### Ventajas:
1. **Rápido**: Se basa en reglas simples y no requiere conocimiento gramatical.
2. **Menor consumo de recursos**: Es computacionalmente eficiente.

#### Desventajas:
1. **Impreciso**: Puede producir raíces que no son palabras reales.
   - Ejemplo: "mejor" → "mej"
2. **Desinformativo**: Pierde matices gramaticales, como tiempo verbal o pluralidad.

#### Implementación en Python:
```python
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("spanish")
palabras = ["corriendo", "corrió", "corre"]
stems = [stemmer.stem(palabra) for palabra in palabras]
print(stems)  # ['corr', 'corr', 'corr']
```

---

### Lemmatización

La **lemmatización** reduce las palabras a su forma base o **lema**, considerando su significado y contexto gramatical. Para lograrlo, utiliza un diccionario que identifica el lema correcto de cada palabra.

#### Ejemplo:
- Palabras: "corriendo", "corrió", "correr"
- Resultado de la lematización: "correr"

#### Ventajas:
1. **Precisa**: Retorna palabras válidas en el idioma.
2. **Contextualizada**: Considera el significado y la función gramatical.

#### Desventajas:
1. **Más lenta**: Requiere análisis morfológico y acceso a diccionarios léxicos.
2. **Mayor consumo de recursos**: Es computacionalmente más costosa.

#### Implementación en Python:
Usando `spaCy`:
```python
import spacy

nlp = spacy.load("es_core_news_sm")
texto = "Los niños están corriendo rápidamente hacia el parque."
doc = nlp(texto)
lemmatized = [token.lemma_ for token in doc]
print(lemmatized)  # ['el', 'niño', 'estar', 'correr', 'rápidamente', 'hacia', 'el', 'parque']
```

---

### Diferencias clave

| Característica      | Stemming                     | Lemmatización                  |
|---------------------|------------------------------|--------------------------------|
| **Base**            | Reglas lingüísticas simples  | Diccionarios léxicos          |
| **Forma resultante**| No siempre es una palabra válida | Siempre retorna una palabra válida |
| **Velocidad**       | Más rápido                  | Más lento                     |
| **Precisión**       | Menor                       | Mayor                         |
| **Uso**             | Análisis rápido y superficial | Tareas que requieren contexto |

---

### ¿Cuándo usar cada uno?

#### **Usa Stemming si...**
1. Necesitas procesamiento rápido.
2. El contexto no es relevante.
3. Estás trabajando con tareas como clustering, donde solo importa la similitud de las raíces.

#### **Usa Lemmatización si...**
1. La precisión es clave.
2. El contexto y la gramática son importantes.
3. Estás trabajando con modelos avanzados o tareas como análisis semántico o traducción.

---

### Ejemplo combinado de Stemming y Lemmatización

```python
from nltk.stem import SnowballStemmer
import spacy

# Inicializar herramientas
stemmer = SnowballStemmer("spanish")
nlp = spacy.load("es_core_news_sm")

# Texto de ejemplo
texto = "Las aves estaban volando por el cielo."

# Aplicar stemming
tokens = texto.split()
stems = [stemmer.stem(token) for token in tokens]
print("Stemming:", stems)

# Aplicar lematización
doc = nlp(texto)
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)
```

Salida:
```
Stemming: ['las', 'aves', 'est', 'vol', 'por', 'el', 'ciel']
Lemmatización: ['el', 'ave', 'estar', 'volar', 'por', 'el', 'cielo']
```

---


In [ ]:
# Inicializar herramientas
stemmer = SnowballStemmer("spanish")
nlp = spacy.load("es_core_news_sm")

# Texto de ejemplo
texto = "Las aves estaban volando por el cielo."

# Aplicar stemming
tokens = texto.split()
stems = [stemmer.stem(token) for token in tokens]
print("Stemming:", stems)



Stemming: ['las', 'aves', 'estab', 'vol', 'por', 'el', 'cielo.']


In [ ]:
############## Stemming ################
from nltk.stem import SnowballStemmer


## Stemmer en español

stemmer = SnowballStemmer('spanish')

## Stemming

texto_stemming = [stemmer.stem(palabra) for palabra in texto_filtrado]

### La clase SnowballStemmer de nltk permite realizar stemming en español, se aplica stemming a las palabras filtradas.

texto_stemming

['hac',
 'vari',
 'años',
 'peloton',
 'fusil',
 'coronel',
 'aurelian',
 'buend',
 'record',
 'aquell',
 'tard',
 'remot',
 'padr',
 'llev',
 'conoc',
 'hiel']

Importante descargar el modelo en español de `spaCy` antes de ejecutar el código:

```bash
!python -m spacy download es_core_news_sm
```

In [ ]:
############## Lematización ################
import spacy
nlp = spacy.load("es_core_news_sm")
## Lematizador en español

# Aplicar lematización
doc = nlp(' '.join(texto_filtrado))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['hacer', 'vario', 'año', 'pelotón', 'fusilamiento', 'coronel', 'aureliano', 'buendía', 'recordar', 'aquel', 'tardar', 'remoto', 'padre', 'llevar', 'conocer', 'helar']


## 7. Bag of words


El modelo **Bag of Words** es una de las técnicas más simples y utilizadas en el procesamiento de lenguaje natural (NLP) para representar texto en un formato numérico que las máquinas pueden entender. A pesar de su simplicidad, sigue siendo una herramienta poderosa para tareas como clasificación de texto, clustering y análisis de sentimiento.

---

### ¿Qué es Bag of Words?

El modelo **BoW** convierte un conjunto de textos (corpus) en una representación matricial basada únicamente en la **frecuencia de las palabras** que aparecen en los documentos, ignorando su orden y contexto. En esencia, crea un "bolso" de palabras donde la posición de las palabras no importa, pero su presencia o ausencia sí.

#### Ejemplo:
Corpus:
1. "El gato duerme"
2. "El perro corre"
3. "El gato corre"

**Bolso de palabras**:
- ["El", "gato", "duerme", "perro", "corre"]

**Matriz de frecuencias**:
| Documento    | El  | gato | duerme | perro | corre |
|--------------|-----|------|--------|-------|-------|
| 1. "El gato duerme" | 1   | 1    | 1      | 0     | 0     |
| 2. "El perro corre" | 1   | 0    | 0      | 1     | 1     |
| 3. "El gato corre"  | 1   | 1    | 0      | 0     | 1     |

---

### ¿Cómo funciona?

1. **Tokenización**:
   - Divide los documentos en palabras (tokens).

2. **Construcción del vocabulario**:
   - Crea una lista de palabras únicas en el corpus.

3. **Vectorización**:
   - Representa cada documento como un vector de frecuencias basado en el vocabulario.

---

### Ventajas

1. **Simplicidad**:
   - Es fácil de entender e implementar.
   
2. **Eficiencia**:
   - Funciona bien con textos cortos y corpora pequeños.

3. **Compatibilidad**:
   - Puede integrarse con modelos tradicionales como Naive Bayes, SVM, o redes neuronales básicas.

---

### Desventajas

1. **Ignora el contexto**:
   - No captura relaciones entre palabras, como sinónimos o frases.
   
2. **Alta dimensionalidad**:
   - Si el corpus tiene muchas palabras únicas, el vector resultante será grande y disperso.

3. **No distingue la importancia de las palabras**:
   - Palabras comunes como "el", "de", "y" tienen el mismo peso que palabras más significativas.

---

### Mejoras al modelo BoW

1. **TF-IDF (Term Frequency-Inverse Document Frequency)**:
   - Ajusta las frecuencias para dar más peso a palabras relevantes y reducir el peso de palabras comunes.
   
2. **N-grams**:
   - Considera secuencias de palabras (como pares o tríos) en lugar de palabras individuales.

3. **Reducción de dimensionalidad**:
   - Usa técnicas como PCA o selección de características para manejar la alta dimensionalidad.

---

### Implementación en Python

#### Usando `CountVectorizer` de `sklearn`
```python
from sklearn.feature_extraction.text import CountVectorizer

# Corpus de ejemplo
corpus = [
    "El gato duerme",
    "El perro corre",
    "El gato corre"
]

# Inicializar el vectorizador
vectorizer = CountVectorizer()

# Transformar el corpus en una matriz BoW
X = vectorizer.fit_transform(corpus)

# Mostrar el vocabulario
print("Vocabulario:", vectorizer.get_feature_names_out())

# Mostrar la matriz BoW
print("Matriz BoW:\n", X.toarray())
```

**Salida**:
```
Vocabulario: ['corre' 'duerme' 'el' 'gato' 'perro']
Matriz BoW:
 [[0 1 1 1 0]
  [1 0 1 0 1]
  [1 0 1 1 0]]
```

---

### ¿Cuándo usar Bag of Words?

1. **Tareas sencillas**:
   - Clasificación de texto, análisis de sentimiento o detección de spam.

2. **Modelos tradicionales**:
   - Funciona bien con Naive Bayes, SVM o regresión logística.

3. **Corpus pequeños**:
   - En textos largos o complejos, su incapacidad para capturar contexto se vuelve un problema.

---

- **Simplicidad**: Fácil de entender y usar.
- **Limitaciones**: Ignora contexto y puede generar vectores grandes.
- **Alternativas**: TF-IDF o embeddings como Word2Vec y BERT para modelos más avanzados.

In [ ]:
############## Bag of words ################

## Vectorizador

vectorizador = CountVectorizer()

## Bolsa de palabras

texto_bow = vectorizador.fit_transform([' '.join(lemmas)])

## Veamos un dataframe con la bolsa de palabras

import pandas as pd

df = pd.DataFrame(texto_bow.toarray(), columns=vectorizador.get_feature_names_out())
df_index = pd.DataFrame([' '.join(lemmas)], columns=['Texto'])

df = pd.concat([df_index, df], axis=1)

df

,Texto,aquel,aureliano,año,buendía,conocer,coronel,fusilamiento,hacer,helar,llevar,padre,pelotón,recordar,remoto,tardar,vario
0,hacer vario año pelotón fusilamiento coronel a...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## 8. Conclusiones

En este notebook, se presentaron los pasos más comunes en el preprocesamiento de texto. Es importante tener en cuenta que estos pasos pueden variar dependiendo del problema y del texto que se esté analizando. Para las siguientes tareas es probable que se hagan algunos de los pasos mencionados anteriormente.

<img src="https://www.mermaidchart.com/raw/b60c8142-12ae-46c9-8412-7333ee1508cd?theme=light&version=v0.1&format=svg">